### Imports

In [1]:
import pandas as pd

### Read data from 'xlsx' file

In [2]:
# Read 'xlsx' file as dataframe
df = pd.read_excel('../data/processed/data_cuba.xlsx', index_col=None, sheet_name='base_limpia')
# Create a list with dataframe column names
cols = df.columns

### Create pivot table from dataframe

In [3]:
# Create a pivot table from df
dilatacion = pd.pivot_table(df, values=cols[0], index=cols[27],
                         columns=cols[81], aggfunc='count')

### Clean data

In [4]:
# Convert index to columns and reset index
dilatacion = dilatacion.rename_axis('Hospital').reset_index().rename_axis('', axis='columns')

# Delete column and rename other column
dilatacion = dilatacion.drop('No', 1).rename(columns={'Sí': 'Dilatación_Manual'})

# Remove rows with NaN values on column
dilatacion.dropna(subset=['Dilatación_Manual'], inplace=True)

# Reset index
dilatacion = dilatacion.reset_index(drop=True)

# Regular expressions to clean data typos
dilatacion['Hospital'].replace(r'\n|^\s', '', regex=True, inplace=True)
dilatacion['Hospital'].replace(r'^\s', '', regex=True, inplace=True)

dilatacion.shape

/tmp/ipykernel_15459/1990314256.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dilatacion = dilatacion.drop('No', 1).rename(columns={'Sí': 'Dilatación_Manual'})


(38, 2)

### Merge two dataframes based on a column

In [5]:
directorio = pd.read_excel(r'../data/processed/directorio_hospitales.xlsx', index_col=None, sheet_name='base_limpia', converters={'Long':str, 'Lat': str})

# Merge dataframes by a column
df_join = pd.merge(dilatacion, directorio, on='Hospital', how='left')

df_join.shape

(38, 4)

In [6]:
# Remove rows with NaN values on column
df_join.dropna(subset=['Long'], inplace=True)
# Reset index
df_join = df_join.reset_index(drop=True)

# Save dataframe as 'xlsx' file
# df_join.to_excel('../data/processed/dilatacion_hospitales.xlsx', index = False)
# df_join.to_json('../data/processed/json/dilatacion_hospitales.json', orient='index', force_ascii=False)

df_join.sample(3)

,Hospital,Dilatación_Manual,Long,Lat
34,Hospital Ginecobstétrico Provincial Ana Betanc...,33.0,21.389169031169487,-77.93928194
9,Hospital Clínico Quirúrgico Docente Julio Trig...,1.0,23.052338527256374,-82.36436551722404
12,Hospital Docente Ginecobstétrico Justo Legón P...,1.0,22.421659519650092,-83.70207667322798
